# OpenRouteService python bindings

https://github.com/GIScience/openrouteservice-py

In [2]:
import openrouteservice as ors
import folium
import json
from pprint import pprint

#povo_center = [46.0682, 11.1551]
#povo_zoom = 16
#povo_start = []

mezzo_center = [46.2147, 11.1065]
mezzo_zoom = 15
mezzo_start = [46.2170,11.1066]

#contains two features MultiPoint
#povo_points = json.load(open('./data/povo.geojson'))['features']
mezzo_points = json.load(open('./data/mezzocorona.geojson'))['features']

center = mezzo_center
zoom = mezzo_zoom
points = mezzo_points
point_start = mezzo_start
point_end = [mezzo_start[0], mezzo_start[1] - 0.005]


# OpenRouteService TSP Routing examples

[API Directions](https://openrouteservice.org/dev/#/api-docs/directions)
[more examples](https://github.com/GIScience/openrouteservice-py/blob/master/examples/basic_example.ipynb)

In [3]:
M = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)

client = ors.Client(base_url='http://ors:8080/ors', key='')

points = {
    'A': points[0]['geometry']['coordinates'],
    'B': points[1]['geometry']['coordinates']
}

routeA = client.directions(
    coordinates = points['A'],
    profile = 'driving-car',
    format = 'geojson',
    validate = False
)

routeB = client.directions(
    coordinates = points['B'],
    profile = 'driving-car',
    format = 'geojson',
    validate = False
)

polyA = [list(reversed(coord)) for coord in routeA['features'][0]['geometry']['coordinates']]
polyB = [list(reversed(coord)) for coord in routeB['features'][0]['geometry']['coordinates']]

folium.PolyLine(polyA, color='red').add_to(M)
folium.PolyLine(polyB, color='blue').add_to(M)
#pprint(points['A'])

for c in points['A']:
    loc = list(reversed(c))
    folium.Marker(loc, icon=folium.Icon(icon='archive', prefix='fa', color='green')).add_to(M)

for c in points['B']:
    loc = list(reversed(c))
    folium.Marker(loc, icon=folium.Icon(icon='archive', prefix='fa', color='green')).add_to(M)

M

## VRP Vehicle Routing Problem 

Optimize job scheduling for multiple vehicles.
The next example makes more use of the power of the ORS optimization endpoint. Two vehicles are assigned to 6 jobs, where the capacity constraints are such that each vehicle can only carry out 3 jobs on its route.

[API Optimization](https://openrouteservice.org/dev/#/api-docs/optimization/post)

In [10]:
m = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)

vehicle_locs = [point_start, point_end]

# posizione di partenza veicoli
vehicles = []
for idx, cc in enumerate(vehicle_locs):
    loc = list(reversed(cc))
    vehicles.append(ors.optimization.Vehicle(
        id = idx,
        profile = 'driving-hgv',
        start = loc,
        end = loc,
        capacity = [3]  # Limit capacity so only 3 jobs can be taken by each vehicle
    ))
    folium.Marker(location=cc, icon=folium.Icon(icon='truck', prefix='fa', color='blue')).add_to(m)


#punti di raccolta
job_locs = points['A'] + points['B']

jobs = []
for idx, cc in enumerate(job_locs):
    loc = list(reversed(cc))
    jobs.append(ors.optimization.Job(
        id = idx,
        location = cc,
        amount = [1] # capacity in vehicle
    ))
    folium.Marker(location=loc, icon=folium.Icon(icon='archive', prefix='fa', color='green')).add_to(m)
m

In [23]:
clientVrp = ors.Client(base_url='http://vroom:8081', key='')

optimized = clientVrp.optimization(
    jobs = jobs,
    vehicles = vehicles,
    geometry = True
)

#optimized['routes'][0]

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(optimized['routes'][0]['geometry'])['coordinates']],
    color='red'
).add_to(m)

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(optimized['routes'][1]['geometry'])['coordinates']],
    color='orange'
).add_to(m)

m

In [29]:
vehicle1 = optimized['routes'][0]
vehicle2 = optimized['routes'][1]

vehicle1['steps']

[{'type': 'start',
  'location': [11.1066, 46.217],
  'arrival': 0,
  'duration': 0,
  'distance': 0},
 {'type': 'job',
  'location': [11.116554737091064, 46.21342719651941],
  'job': 5,
  'service': 0,
  'waiting_time': 0,
  'arrival': 116,
  'duration': 116,
  'distance': 1260},
 {'type': 'job',
  'location': [11.118743419647217, 46.21494168708413],
  'job': 4,
  'service': 0,
  'waiting_time': 0,
  'arrival': 150,
  'duration': 150,
  'distance': 1558},
 {'type': 'job',
  'location': [11.114044189453129, 46.21506046889148],
  'job': 3,
  'service': 0,
  'waiting_time': 0,
  'arrival': 181,
  'duration': 181,
  'distance': 1933},
 {'type': 'end',
  'location': [11.1066, 46.217],
  'arrival': 234,
  'duration': 234,
  'distance': 2561}]

In [27]:
vehicle2['steps']

[{'type': 'start',
  'location': [11.1016, 46.217],
  'arrival': 0,
  'duration': 0,
  'distance': 0},
 {'type': 'job',
  'location': [11.09722137451172, 46.21403596735467],
  'job': 2,
  'service': 0,
  'waiting_time': 0,
  'arrival': 109,
  'duration': 109,
  'distance': 732},
 {'type': 'job',
  'location': [11.092135906219482, 46.216560067112056],
  'job': 1,
  'service': 0,
  'waiting_time': 0,
  'arrival': 214,
  'duration': 214,
  'distance': 1345},
 {'type': 'job',
  'location': [11.095612049102783, 46.21736181569706],
  'job': 0,
  'service': 0,
  'waiting_time': 0,
  'arrival': 275,
  'duration': 275,
  'distance': 1812},
 {'type': 'end',
  'location': [11.1016, 46.217],
  'arrival': 331,
  'duration': 331,
  'distance': 2284}]